In [ ]:
%matplotlib inline
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10,5)})
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
import wget
import datetime

pd.set_option('display.float_format', lambda x: '%.1f' % x)

%config InlineBackend.figure_format = 'svg'

In [ ]:
t = yf.Ticker("CL=F")

In [ ]:
t = yf.Ticker("BTC-USD")

In [ ]:
t.hist = t.history(period="max")

In [ ]:
t.history(period='max').Close.plot();

In [ ]:
url_eu = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01239&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"

url_usd = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01235&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"

#R01239 евро 
#R01235 доллар

In [ ]:
curs_eu = pd.read_excel(wget.download(url_eu))
curs_usd = pd.read_excel(wget.download(url_usd))

# Задание 1.

Вы - менеджер по продажам в нефтехимической компании. Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании. Ваша компания запускает новый продукт - "Волшебный белый порошок" (ВБП), который по своим свойствам похож на тот, что уже есть на рынке. Ваши клиенты предпочитают формульный подход к ценообразованию. Вам нужно предложить клиентам формулу расчета цены на ВБП. В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.

### Дополнительные вводные:

Продукт производиться из нефти. Один из известных способов производства предполагает что из 16 баррелей нефти сделают одну тонну ВБП, а затраты на производство в районе 400 евро.

In [ ]:
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Monty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Poly':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'up to 100': 0.01, # 1%
             'up to 300': 0.05, # 5%
             '300 plus': 0.1}   #10%


### Что нужно сделать?

1. Проанализировать имеющиеся данные.
2. Определить базовую формулу цены на условиях FCA (цена на заводе). То есть как бы выглядела цена на исторических данных.
3. Отобразить на графике.
4. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
5. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить  файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента… 

### В результате должно быть 3 файла:
1. Ноутбук с расчетами и графиками. 
2. Скрипт, который делает расчет цены для каждого клиента в одном файле
2. Скрипт, который делает расчет цены для каждого клиента в отдельном файле

In [ ]:
def discount(x):
    
    if x['volumes'] <= 100:
        result = 0.01
        
    elif (x['volumes'] > 100 and x['volumes'] <= 300):
        result = 0.05
        
    elif x['volumes'] > 300:
        result = 0.1

    return result

In [ ]:
def process_data():
    df = pd.DataFrame(customers)
    df = df.T.reset_index().rename(columns = {'index':'clientName'})
    df['nominal'] = 1

    df = pd.merge(df, curs_eu)
    df = pd.merge(df, curs_usd, on = ['nominal', 'data'], suffixes=('_eu', '_usd'))

    df['discount'] = df.apply(lambda row : discount(row), axis=1) 

    df['logistics_costs'] = df.loc[df['location'] == 'EU', 'logistics_costs'] = 30
    df.loc[df['location'] == 'CN', 'logistics_costs'] = 130

    df['volumes'] = df['volumes'] * 16
    
    
    
    return df

In [ ]:
def calculation():
    
    df = process_data()
    df1 = df.query("location == 'EU'").copy()
    df1['bill'] = (df1['volumes'] * 400 * df1['curs_eu'] +\
        df1['volumes'] * df1['logistics_costs'] * df1['curs_eu']) * (1 - df1['discount'])

    df2 = df.query("location == 'CN'").copy()
    df2['bill'] = (df2['volumes'] * 400 * df2['curs_eu'] +\
        df2['volumes'] * df2['logistics_costs'] * df2['curs_usd']) * (1 - df2['discount'])

    df = pd.concat([df1, df2])
    df = df.astype({'bill':'float'})
    
    return df

In [ ]:
def main():
    
    df = calculation()
    df.to_excel('./data.xlsx', index=False)

    for cl in df.clientName.unique():
        df.query(f"clientName == '{cl}'").sort_values(by='data').to_excel('./'+cl+'_data.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
sns.lineplot(data=df, x="data", y="bill", hue='clientName');


# Задание 2. 

Клиенты согласились на ваше предложение. 
Вам нужно сделать написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно. 
Важный момент: данные для расчета скрипт должен подтягивать сам.

В результате должно быть 3 файла
1. Ноутбук с расчетами и графиками (чтобы данные подтягивались туда)
2. Скрипт, который делает расчет цены для клиента (чтобы данные подтягивались туда сами)

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10,5)})
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
import wget
pd.set_option('display.float_format', lambda x: '%.1f' % x)
%config InlineBackend.figure_format = 'svg'

#CONSTANTS
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
CUSTOMERS = {
    'Monty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Poly':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
DISCOUNTS = {'up to 100': 0.01, # 1%
             'up to 300': 0.05, # 5%
             '300 plus': 0.1}   #10%

def get_eu_data():
    url_eu = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01239&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"

    curs_eu = pd.read_excel(wget.download(url_eu))
    
    return curs_eu

def get_usd_data():

    url_usd = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01235&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"
    curs_usd = pd.read_excel(wget.download(url_usd))
    
    return curs_usd

def discount(x):
    
    if x['volumes'] <= 100:
        result = list(DISCOUNTS.values())[0]
        
    elif (x['volumes'] > 100 and x['volumes'] <= 300):
        result = list(DISCOUNTS.values())[1]
        
    elif x['volumes'] > 300:
        result = list(DISCOUNTS.values())[2]

    return result

def process_data():
    df = pd.DataFrame(CUSTOMERS)
    df = df.T.reset_index().rename(columns = {'index':'clientName'})
    df['nominal'] = 1
    
    curs_eu = get_eu_data()
    curs_usd = get_usd_data()

    df = pd.merge(df, curs_eu)
    df = pd.merge(df, curs_usd, on = ['nominal', 'data'], suffixes=('_eu', '_usd'))

    df['discount'] = df.apply(lambda row : discount(row), axis=1) 

    df['logistics_costs'] = df.loc[df['location'] == 'EU', 'logistics_costs'] = 30
    df.loc[df['location'] == 'CN', 'logistics_costs'] = 130

    df['volumes'] = df['volumes'] * 16
    
    df['month'] = df['data'].dt.strftime('%b')
    
    return df


def calculation():
    
    df = process_data()
    df1 = df.query("location == 'EU'").copy()
    df1['bill'] = (df1['volumes'] * 400 * df1['curs_eu'] +\
        df1['volumes'] * df1['logistics_costs'] * df1['curs_eu']) * (1 - df1['discount'])

    df2 = df.query("location == 'CN'").copy()
    df2['bill'] = (df2['volumes'] * 400 * df2['curs_eu'] +\
        df2['volumes'] * df2['logistics_costs'] * df2['curs_usd']) * (1 - df2['discount'])

    df = pd.concat([df1, df2])
    df = df.astype({'bill':'float'})
    
    return df

def main():
    
    df = calculation()
    df.to_excel('./data.xlsx', index=False)

    for cl in df.clientName.unique():
        df.query(f"clientName == '{cl}'").sort_values(by='data').to_excel('./'+cl+'_data.xlsx', index=False)

        
if __name__ == '__main__':
    main()


# Задание 3. 
К вам пришел новый клиент из России. Который использует продукт А и готов регулярно закупать ВБП для того, чтобы не быть привязанным к одному поставщику. Клиент готов забирать ВБП с завода самостоятельно.
1. На основе тех данных, которые уже есть предложите цену новому клиенту. 
2. Подготовьте скрипт, который будет использовать курс публикуемый на <a href='http://www.cbr.ru/currency_base/dynamics/'> сайте ЦБ РФ</a>.


In [ ]:
%%writefile myscript.py


import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10,5)})
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
import wget
pd.set_option('display.float_format', lambda x: '%.1f' % x)
%config InlineBackend.figure_format = 'svg'

#CONSTANTS
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# * Справочная информация по клиентам(объемы, локации, комментарии) 
CUSTOMERS = {
    'Tariff 1':{
        'location':'RU_1',
        'volumes':400,
        'comment':'monthly'
    },
    
    'Tariff 2':{
        'location':'RU_2',
        'volumes': 200,
        'comment': 'monthly'
    },
    'Tariff 3':{
        'location':'RU_3',
        'volumes': 90,
        'comment': 'monthly'
    }
}
# Скидки
DISCOUNTS = {'up to 100': 0.01, # 1%
             'up to 300': 0.05, # 5%
             '300 plus': 0.1}   #10%

def get_eu_data():
    url_eu = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01239&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"

    curs_eu = pd.read_excel(wget.download(url_eu))
    
    return curs_eu

def get_usd_data():

    url_usd = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01235&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"
    curs_usd = pd.read_excel(wget.download(url_usd))
    
    return curs_usd

def discount(x):
    
    if x['volumes'] <= 100:
        result = list(DISCOUNTS.values())[0]
        
    elif (x['volumes'] > 100 and x['volumes'] <= 300):
        result = list(DISCOUNTS.values())[1]
        
    elif x['volumes'] > 300:
        result = list(DISCOUNTS.values())[2]

    return result

def process_data():
    df = pd.DataFrame(CUSTOMERS)
    df = df.T.reset_index().rename(columns = {'index':'tariff'})
    df['nominal'] = 1
    df = df.astype({'volumes':'int'})
    
    curs_eu = get_eu_data()
    curs_usd = get_usd_data()

    df = pd.merge(df, curs_eu)
    df = pd.merge(df, curs_usd, on = ['nominal', 'data'], suffixes=('_eu', '_usd'))

    df['discount'] = df.apply(lambda row : discount(row), axis=1) 

    df['volumes'] = df['volumes'] * 16
    
    df['month'] = df['data'].dt.strftime('%b')
    
    return df


def calculation():
    
    df = process_data()
    df['bill'] = df['volumes'] * 400 * df['curs_eu'] * (1 - df['discount'])
    
    return df

def main():
    
    df = calculation()
    df.to_excel('./data.xlsx', index=False)

if __name__ == '__main__':
    main()


# Задание 4. 

* Подумайте, как можно проверить корректность работы алгоритма? 
* Какие возможны ошибки?
* Придумайте тесты для проверки возможных ошибок алгоритма
* Интергрируйте их в свои скрипты в виде исключений


In [ ]:
%%writefile myscript.py


import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10,5)})
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
import wget
pd.set_option('display.float_format', lambda x: '%.1f' % x)
#%config InlineBackend.figure_format = 'svg'

#CONSTANTS
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# * Справочная информация по клиентам(объемы, локации, комментарии) 
CUSTOMERS = {
    'Tariff 1':{
        'location':'RU_1',
        'volumes':400,
        'comment':'monthly'
    },
    
    'Tariff 2':{
        'location':'RU_2',
        'volumes': 200,
        'comment': 'monthly'
    },
    'Tariff 3':{
        'location':'RU_3',
        'volumes': 90,
        'comment': 'monthly'
    }
}
# Скидки
DISCOUNTS = {'up to 100': 0.01, # 1%
             'up to 300': 0.05, # 5%
             '300 plus': 0.1}   #10%



class CustomError(Exception):
    pass

def get_eu_data():
    
    try:
        url_eu = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01239&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"

        curs_eu = pd.read_excel(wget.download(url_eu))
        
    except:
        raise CustomError('Ошибка. Не удалось получить данные.')
    
    return curs_eu

def get_usd_data():

    try:
        url_usd = "http://cbr.ru/Queries/UniDbQuery/DownloadExcel/98956?Posted=True\
&so=1&mode=1&VAL_NM_RQ=R01235&From=01.03.2010To=26.03.2022&FromDate=03%2F01%2F2010&ToDate=03%2F26%2F2022"
        curs_usd = pd.read_excel(wget.download(url_usd))
    except:
        raise CustomError('Ошибка. Не удалось получить данные.')
        
    return curs_usd

def discount(x):
    
    if x['volumes'] <= 100:
        result = list(DISCOUNTS.values())[0]
        
    elif (x['volumes'] > 100 and x['volumes'] <= 300):
        result = list(DISCOUNTS.values())[1]
        
    elif x['volumes'] > 300:
        result = list(DISCOUNTS.values())[2]

    return result

def process_data():
    
    df = pd.DataFrame(CUSTOMERS)
    
    df = df.T.reset_index().rename(columns = {'index':'tariff'})
    df['nominal'] = 1
    
    try:
        df = df.astype({'volumes':'int'})
    except:
        raise CustomError('Ошибка. В таблице есть элементы с неверным типом данных или пропуски.')
    
    curs_eu = get_eu_data()
    curs_usd = get_usd_data()

    df = pd.merge(df, curs_eu)
    df = pd.merge(df, curs_usd, on = ['nominal', 'data'], suffixes=('_eu', '_usd'))

    df['discount'] = df.apply(lambda row : discount(row), axis=1) 

    df['volumes'] = df['volumes'] * 16
    
    df['month'] = df['data'].dt.strftime('%b')
    
    return df


def calculation():
    
    df = process_data()
    df['bill'] = df['volumes'] * 400 * df['curs_eu'] * (1 - df['discount'])
    
    return df

def main():
    
    df = calculation()
    df.to_excel('./data.xlsx', index=False)

if __name__ == '__main__':
    main()


# Задание 5.

Вы решили пойти в отпуск на время вашего отсутствия вас будет заменять коллега. Он будет заниматься рассчетом цен для клиентов. Чтобы ему было проще вам нужно завернуть ваши скрипты в модуль, который будет запускать у себя коллега.

* Создайте модуль, который можно запускать в ноутбуке подкладывая в ту же директорию
* Хорошенько задокументируйте ВСЁ, но помните о PEP8

In [ ]:
# import yourpackage
import myscript

volumes_script.main

# Задание 6. 

Ваши коллеги узнали, что вы быстро рассчитываете цены для формульных контрактов, и решили использовать такую же практику для своих продуктов и клиентов. Вам нужно создать пакет, чтобы его было проще устанавливать и использовать.

* Создайте отдельный пакет, который можно устанавливать с помощью pip 
* В пакете должны быть реализованы функции для рассчета цен по формульным контрактам
* Выложите пакет в репозиторий

In [ ]:
!pip install myscript
import myscript